In [3]:
import pandas as pd
from collections import deque
from datetime import timedelta


class Parcel:
    def __init__(self, parcel_id, arrival_time, feasible_outfeeds):
        self.id = parcel_id
        self.arrival_time = arrival_time
        self.feasible_outfeeds = feasible_outfeeds
        self.sorted = False
        self.recirculated = False


class PosiSorterSystem:
    def __init__(self, layout_df, outfeed_capacity=3):
        self.belt_speed = layout_df.loc[layout_df['Layout property'] == 'Belt Speed', 'Value'].values[0]
        self.num_outfeeds = 3  # Aangenomen op basis van de Excel
        self.outfeed_capacity = outfeed_capacity
        self.outfeeds = [deque(maxlen=outfeed_capacity) for _ in range(self.num_outfeeds)]
        self.recirculation_belt = deque()
        self.processed_parcels = []

    def sort_parcel(self, parcel):
        for outfeed_index in parcel.feasible_outfeeds:
            if len(self.outfeeds[outfeed_index]) < self.outfeed_capacity:
                self.outfeeds[outfeed_index].append(parcel)
                parcel.sorted = True
                print(f"Pakket {parcel.id} gesorteerd naar outfeed {outfeed_index}")
                return
        parcel.recirculated = True
        self.recirculation_belt.append(parcel)
        print(f"Pakket {parcel.id} kon niet gesorteerd worden en werd gerecirculeerd.")

    def remove_from_outfeeds(self):
        for i, outfeed in enumerate(self.outfeeds):
            if outfeed:
                removed = outfeed.popleft()
                print(f"Pakket {removed.id} verwijderd uit outfeed {i}")

    def run_simulation(self, parcels):
        current_time = parcels[0].arrival_time
        end_time = parcels[-1].arrival_time
        parcel_index = 0

        while current_time <= end_time:
            print(f"\n--- Tijdstip: {current_time.time()} ---")

            # Verwerk nieuwe aankomsten
            while parcel_index < len(parcels) and parcels[parcel_index].arrival_time <= current_time:
                parcel = parcels[parcel_index]
                parcel_index += 1
                self.processed_parcels.append(parcel)
                print(f"Pakket {parcel.id} aangekomen – haalbare outfeeds: {parcel.feasible_outfeeds}")
                self.sort_parcel(parcel)

            # Simuleer verwijdering van pakketten
            self.remove_from_outfeeds()
            current_time += timedelta(seconds=1)

        self.print_summary()

    def print_summary(self):
        total = len(self.processed_parcels)
        sorted_count = sum(p.sorted for p in self.processed_parcels)
        recirculated_count = sum(p.recirculated for p in self.processed_parcels)
        print("\n--- Simulatie Samenvatting ---")
        print(f"Totaal aantal verwerkte pakketten: {total}")
        print(f"Aantal gesorteerd: {sorted_count}")
        print(f"Aantal gerecirculeerd: {recirculated_count}")
        print(f"Succesratio sorteren: {sorted_count / total:.2%}")


def load_parcels_from_excel(parcels_df):
    parcels = []
    for _, row in parcels_df.iterrows():
        parcel_id = int(row['Parcel Number'])
        arrival_time = pd.to_datetime(row['Arrival Time'])
        feasible_outfeeds = [i for i, flag in enumerate([row['Outfeed 1'], row['Outfeed 2'], row['Outfeed 3']]) if flag]
        parcels.append(Parcel(parcel_id, arrival_time, feasible_outfeeds))
    return sorted(parcels, key=lambda p: p.arrival_time)


def main():
    # Excel inladen
    xls = pd.ExcelFile("PosiSorterData1.xlsx")
    parcels_df = xls.parse('Parcels')
    layout_df = xls.parse('Layout')

    # Data verwerken
    parcels = load_parcels_from_excel(parcels_df)
    system = PosiSorterSystem(layout_df)
    system.run_simulation(parcels)


if __name__ == "__main__":
    main()




--- Tijdstip: 09:00:00 ---
Pakket 16 aangekomen – haalbare outfeeds: [2]
Pakket 16 gesorteerd naar outfeed 2
Pakket 133 aangekomen – haalbare outfeeds: [2]
Pakket 133 gesorteerd naar outfeed 2
Pakket 149 aangekomen – haalbare outfeeds: [2]
Pakket 149 gesorteerd naar outfeed 2
Pakket 16 verwijderd uit outfeed 2

--- Tijdstip: 09:00:01 ---
Pakket 133 verwijderd uit outfeed 2

--- Tijdstip: 09:00:02 ---
Pakket 1 aangekomen – haalbare outfeeds: [0]
Pakket 1 gesorteerd naar outfeed 0
Pakket 1 verwijderd uit outfeed 0
Pakket 149 verwijderd uit outfeed 2

--- Tijdstip: 09:00:03 ---

--- Tijdstip: 09:00:04 ---
Pakket 2 aangekomen – haalbare outfeeds: [0, 2]
Pakket 2 gesorteerd naar outfeed 0
Pakket 2 verwijderd uit outfeed 0

--- Tijdstip: 09:00:05 ---

--- Tijdstip: 09:00:06 ---

--- Tijdstip: 09:00:07 ---

--- Tijdstip: 09:00:08 ---

--- Tijdstip: 09:00:09 ---

--- Tijdstip: 09:00:10 ---

--- Tijdstip: 09:00:11 ---

--- Tijdstip: 09:00:12 ---

--- Tijdstip: 09:00:13 ---

--- Tijdstip: 09:00